In [1]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [4]:
df = pd.read_csv("train (1).csv")

In [5]:
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [6]:
with open("label_num_to_disease_map.json", "r") as file:
    label_name = json.load(file)
label_name

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [8]:
image_path = "Dataset_plant"
image_list = os.listdir(image_path)

In [9]:
# Matching labels with images
df = df[df["image_id"].isin(image_list)]
df.reset_index(drop=True, inplace=True)

In [10]:
path_name = "Dataset_plant"
df["image_path"] = df["image_id"].apply(lambda x: str(path_name+x))
df["label_name"] = df["label"].apply(lambda x: label_name.get(str(x), "Unknown"))
df.head()

C:\Users\ranan\AppData\Local\Temp\ipykernel_580\3253080392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["image_path"] = df["image_id"].apply(lambda x: str(path_name+x))
C:\Users\ranan\AppData\Local\Temp\ipykernel_580\3253080392.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label_name"] = df["label"].apply(lambda x: label_name.get(str(x), "Unknown"))


,image_id,label,image_path,label_name
0,760745341.jpg,2,Dataset_plant760745341.jpg,Cassava Green Mottle (CGM)
1,76080816.jpg,3,Dataset_plant76080816.jpg,Cassava Mosaic Disease (CMD)
2,760863006.jpg,2,Dataset_plant760863006.jpg,Cassava Green Mottle (CGM)
3,760916412.jpg,2,Dataset_plant760916412.jpg,Cassava Green Mottle (CGM)
4,761160675.jpg,3,Dataset_plant761160675.jpg,Cassava Mosaic Disease (CMD)


In [11]:
from sklearn.model_selection import train_test_split

# train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50, VGG19, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input


In [13]:
def build_model(input_shape=(224, 224, 3), num_classes=5):
    # ResNet50 model
    resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x_resnet = resnet50_base.output
    x_resnet = GlobalAveragePooling2D()(x_resnet)

    # VGG19 model
    vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    x_vgg = vgg19_base.output
    x_vgg = GlobalAveragePooling2D()(x_vgg)

    # InceptionV3 model
    inception_input_shape = (299, 299, 3)
    inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=inception_input_shape)
    x_inception = inceptionv3_base.output
    x_inception = GlobalAveragePooling2D()(x_inception)

    # Concatenate the outputs
    combined = Concatenate()([x_resnet, x_vgg, x_inception])

    # Add a dense layer and the final output layer
    x = Dense(1024, activation='relu')(combined)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=[resnet50_base.input, vgg19_base.input, inceptionv3_base.input], outputs=output)
    
    return model


In [14]:
def generate_data(df, batch_size):
    while True:
        for start in range(0, len(df), batch_size):
            x_batch_resnet = []
            x_batch_vgg = []
            x_batch_inception = []
            y_batch = []
            end = min(start + batch_size, len(df))
            df_batch = df[start:end]
            for _, row in df_batch.iterrows():
                img_path = row['image_path']
                
                img_resnet = image.load_img(img_path, target_size=(224, 224))
                img_resnet = image.img_to_array(img_resnet)
                img_resnet = np.expand_dims(img_resnet, axis=0)
                img_resnet = resnet_preprocess(img_resnet)

                img_vgg = image.load_img(img_path, target_size=(224, 224))
                img_vgg = image.img_to_array(img_vgg)
                img_vgg = np.expand_dims(img_vgg, axis=0)
                img_vgg = vgg_preprocess(img_vgg)

                img_inception = image.load_img(img_path, target_size=(299, 299))
                img_inception = image.img_to_array(img_inception)
                img_inception = np.expand_dims(img_inception, axis=0)
                img_inception = inception_preprocess(img_inception)
                
                x_batch_resnet.append(img_resnet[0])
                x_batch_vgg.append(img_vgg[0])
                x_batch_inception.append(img_inception[0])
                
                y_batch.append(row['label'])

            x_batch_resnet = np.array(x_batch_resnet)
            x_batch_vgg = np.array(x_batch_vgg)
            x_batch_inception = np.array(x_batch_inception)
            y_batch = tf.keras.utils.to_categorical(np.array(y_batch), num_classes=len(classes))
            
            yield [x_batch_resnet, x_batch_vgg, x_batch_inception], y_batch


In [15]:
stacked_model = build_model()
stacked_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [16]:
batch_size = 32

train_generator = generate_data(train_df, batch_size)
test_generator = generate_data(test_df, batch_size)

nb_train_samples = len(train_df)
nb_test_samples = len(test_df)


In [18]:
from tensorflow.keras.preprocessing import image


In [ ]:
epochs = 20

stacked_model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size,
    epochs=epochs
)